# Introduction

[insert intro]


In [ ]:
%pip install pandas
%pip install matplotlib

In [18]:
import math
import pandas as pd
import matplotlib.pyplot as plt

# Constants
g = 9.81                        # gravity [m/s^2]
rho = 1000                      # density of water [kg/m^3]
mu = 0.001                      # dynamic viscosity [Pa·s]
kv = 1.004e-6                   # kinematic viscosity [m^2/s]

eps = 1.5e-6                    # pipe roughness [m] (smooth plastic)
D = 0.00794                     # pipe diameter [m]
A = math.pi * (D / 2) ** 2      # pipe cross-sectional area [m^2]
tank_length = 0.32              # [m]
tank_width = 0.26               # [m]
A_t = tank_length * tank_width  # [m^2]
tolerance = 0.001               # tolerance to converge within
K = 0.75                         # minor losses

In [21]:
def compute_velocity(f, h0, L):
    """Computes velocity based on current friction factor f."""
    return math.sqrt((2 * g * h0) / (1 + f * L / D + K))


def compute_reynolds(v):
    """Returns Reynolds number."""
    return v * D / kv

def compute_friction_factor(Re):
    """Returns Darcy friction factor"""
    # Laminar flow
    if Re < 2300:
        return 64 / Re  
    # Transitional: average of laminar and turbulent
    elif Re <= 4000:
        f_lam = 64 / Re
        f_turb = 0.25 / (math.log10(eps / (3.7 * D) + 5.74 / Re**0.9))**2
        return (f_lam + f_turb) / 2
    else:
        # Turbulent
        return 0.25 / (math.log10(eps / (3.7 * D) + 5.74 / Re**0.9))**2
    
def compute_drain_time(L):
    """
    Computes the drain time for a given tube length L (in meters).
    Returns: (time in sec, average velocity, Reynolds number, friction factor)
    """
    # Compute height range based on length
    h0 = 0.1 + L / 150     # initial water height [m]
    hf = 0.02 + L / 150    # final water height [m]
    sqrt_h_diff = math.sqrt(h0) - math.sqrt(hf)

    # Initial guess for friction factor
    f = 0.014
    tolerance = 1e-3

    while True:
        v = compute_velocity(f, h0, L)
        Re = compute_reynolds(v)

        f_new = compute_friction_factor(Re)

        if abs(f_new - f) < tolerance:
            break
        f = f_new

    # Final drain time
    T = (2 * A_t / A) * math.sqrt((1 + f * L / D + K) / (2 * g)) * sqrt_h_diff
    return round(T, 2), round(v, 3), int(Re), round(f, 5)

In [23]:
cases = [
    {"label": "L = 20 cm", "L": 0.20},
    {"label": "L = 30 cm", "L": 0.30},
    {"label": "L = 40 cm", "L": 0.40},
    {"label": "L = 60 cm", "L": 0.60},
    # {"label": "L = 40 cm + T-joint", "L": 0.40},
]

# Run and collect results
results = []
for case in cases:
    T, v, Re, f = compute_drain_time(case["L"])
    results.append({
        "Case": case["label"],
        "Length (m)": case["L"],
        "Drain Time (min:sec)": f"{int(T // 60)}:{int(T % 60):02d}",
        "Velocity (m/s)": v,
        "Re": Re,
        "f": f
    })

# Convert to DataFrame
df = pd.DataFrame(results)
display(df)


,Case,Length (m),Drain Time (min:sec),Velocity (m/s),Re,f
0,L = 20 cm,0.2,3:31,0.871,6885,0.03465
1,L = 30 cm,0.3,3:48,0.805,6367,0.03539
2,L = 40 cm,0.4,4:03,0.752,5943,0.03606
3,L = 60 cm,0.6,4:31,0.669,5287,0.03724


# Conclusion

[insert results and conclusion]
